In [219]:
import sys
import os
script_dir = os.getcwd() # coleata o diretorio do projeto atual
sys.path.append(script_dir)

In [220]:
import pandas as pd
import numpy as np
import nltk
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import models, layers
from keras.models import clone_model
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from copy import deepcopy

In [221]:
data = pd.read_csv(r'CSV\tudo_junto_pp.csv')
data.head(5)

,Ano,Enunciado_Alternativas,Area_de_Conhecimento
0,2020,bienalsur una invitación cruz la front ded señ...,Linguagens
1,2020,españ corazón feder dim una gran sorpr hab pre...,Linguagens
2,2020,dispon httpinversorsaludcom acess 18 ago 2017 ...,Linguagens
3,2020,lo orígen habit expresíon che hay alg má argen...,Linguagens
4,2020,habl con acent andaluz telediari la 9 hac uno ...,Linguagens


In [222]:
areas = data['Area_de_Conhecimento'].unique()
areas

array(['Linguagens', 'Ciências Humanas', 'Ciências da Natureza',
       'Matemática'], dtype=object)

In [223]:
# Passo 1: Defina um mapeamento personalizado
mapa_personalizado = {
    'Linguagens': 0,
    'Ciências Humanas': 1,
    'Ciências da Natureza': 2,
    'Matemática': 3
}

# Passo 2: Substitua os valores na coluna usando o mapeamento
data['Area_de_Conhecimento'] = data['Area_de_Conhecimento'].map(mapa_personalizado)

In [224]:
areas = data['Area_de_Conhecimento'].unique()
areas

array([0, 1, 2, 3], dtype=int64)

In [225]:
# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(data['Enunciado_Alternativas'], data['Area_de_Conhecimento'], test_size=0.2, random_state=1)
X_validation, y_validation = data['Enunciado_Alternativas'], data['Area_de_Conhecimento']

In [226]:
# Vetorização dos textos usando TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()
X_validation = vectorizer.transform(X_validation).toarray()


In [227]:
# Define the MLP Model using PyTorch
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, dense_layers):
        super(MLP, self).__init__()
        layers = []
        
        # Add the first layer
        layers.append(nn.Linear(input_dim, dense_layers[0]))
        layers.append(nn.ReLU())
        
        # Add hidden layers
        for i in range(1, len(dense_layers)):
            layers.append(nn.Linear(dense_layers[i-1], dense_layers[i]))
            layers.append(nn.ReLU())
        
        # Add output layer
        layers.append(nn.Linear(dense_layers[-1], output_dim))
        layers.append(nn.Softmax(dim=1))  # Softmax for multi-class classification
        
        # Stack layers into a Sequential model
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Genetic Algorithm class for optimizing MLP
class GeneticAlgorithm:
    def __init__(self, tamanho_populacao: int, numero_prog: int, alpha: float, MLPdim: tuple, data: tuple) -> None:
        self.tamanho_populacao = tamanho_populacao
        self.numero_prog = numero_prog
        self.alpha = alpha
        self.MLPdim = MLPdim
        self.populacao = []

        # Convert the data into tensors
        self.X_train = torch.tensor(data[0], dtype=torch.float32)
        self.Y_train = torch.tensor(data[1], dtype=torch.long)  # assuming categorical labels
        self.X_test = torch.tensor(data[2], dtype=torch.float32)
        self.Y_test = torch.tensor(data[3], dtype=torch.long)

        self.iniciar_gen()
    
    def iniciar_gen(self) -> None:
        for _ in range(self.tamanho_populacao):
            model = MLP(self.MLPdim[0], self.MLPdim[1], self.MLPdim[2])
            self.populacao.append([model, 0])

    def pontuador(self, resposta_prevista: torch.Tensor, resposta_real: int) -> int:
        return 1 if resposta_prevista == resposta_real else 0
    
    def avaliador(self, individuo) -> int:
        pontuacao = 0
        individuo[0].eval()
        with torch.no_grad():
            resposta_prevista = individuo[0](self.X_train)
            resposta_prevista = torch.argmax(resposta_prevista, dim=1)
        
        for i, pred in enumerate(resposta_prevista):
            pontuacao += self.pontuador(pred.item(), self.Y_train[i].item())
        
        return pontuacao
    
    def avaliador_geracional(self) -> None:
        for index, individuo in enumerate(self.populacao):
            self.populacao[index][1] = self.avaliador(individuo)

    def seleciona_progenitores(self) -> list:
        redes_branch_ordenada = sorted(self.populacao, key=lambda item: item[1], reverse=True)
        return redes_branch_ordenada[:self.numero_prog]

    def gerador_descendentes(self) -> None:
        lista_pais = self.seleciona_progenitores()
        descendentes = self.tamanho_populacao - self.numero_prog
        prox_gen = []

        for x in range(descendentes):
            prox_gen.append(deepcopy(lista_pais[x % len(lista_pais)]))
        for indiv in range(len(prox_gen)):
            prox_gen[indiv][0] = self.mutacao(prox_gen[indiv][0])
        prox_gen.extend(lista_pais)
        self.populacao = prox_gen
    
    def mutacao(self, individuo):
        for param in individuo.parameters():
            if param.requires_grad:
                param.data += self.alpha * torch.randn(param.shape)
        return individuo

    def estrategia_evolutiva(self, n_geracoes: int) -> list:
        for _ in range(n_geracoes):
            self.avaliador_geracional()
            self.gerador_descendentes()
            avaliador = sorted(self.populacao, key=lambda item: item[1], reverse=True)
            print([x[1] for x in avaliador])
            print(len(avaliador))
            
        return self.populacao[:1]


In [228]:
y_train = [x for x in y_train]
y_test = [x for x in y_test]

In [229]:
# self, tamanho_populacao : int, numero_prog : int, alpha : float, MLPdim : tuple[3], data : tuple[4]

tamanho_populacao = 1000
numero_prog = 20
alpha = 10
rede_neural = (X_train.shape[1], len(areas), [64, 32, 64])
data = (X_train, y_train, X_test, y_test)

algoritimo_genetico = GeneticAlgorithm(tamanho_populacao, numero_prog, alpha, rede_neural, data)

KeyboardInterrupt: 

In [218]:
algoritimo_genetico.estrategia_evolutiva(100)

KeyboardInterrupt: 